In [69]:
import re,csv,os,sys,urllib,requests,string
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup

In [153]:
# 爬蟲wiki的英文縮寫統整表table
def getWikiTable():
    # Wikipedia:List of English contractions (https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions)
    # https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722
    website_url = requests.get('https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions').text
    soup = BeautifulSoup(website_url,'lxml')
    table = soup.find('table',{'class':'wikitable sortable'})
    table_td = table.find_all('td')

    # 抓網頁裡的tabe
    L_contraction = []
    L_meaning = []
    for num in range(int(len(table_td)/2)):
        contraction = table_td[2*num].text.replace(' (rarely, cain\'t)','')
        meaning = table_td[2*num+1].text.replace('\n','').replace('[1]','').replace('[2]','').replace('[3]','').replace('[4]','').replace(' (colloquial)','').replace(' (archaic)','').replace(' (colloquial/Southern American English)','').replace(' (possessive forms of many nouns are homographic to this contraction)','')
        L_contraction.append(contraction)
        L_meaning.append(meaning)

    df_contraction = pd.DataFrame(list(zip(L_contraction,L_meaning)),columns=['Contraction','Meaning'])
    df_contraction.to_excel('table/df_contraction_original.xlsx',index=0)
    return df_contraction

In [154]:
df_contraction = getWikiTable()
df_contraction

,Contraction,Meaning
0,'aight,alright
1,ain't,am not / is not / are not / has not / have not...
2,amn't,am not
3,aren't,are not
4,can't,cannot
...,...,...
140,you'll,you shall / you will
141,you're,you are/ you are
142,you've,you have
143,noun's,noun is


In [157]:
# 添加兩個欄位，Contraction2：首字大寫，Final：「'」取代為空白
def editWikiTable(df_contraction):
    L_cap_x = []
    L_cap_x2 = []
    for i in range(len(df_contraction)):
        x = df_contraction['Contraction'][i]
        cap_x = string.capwords(x) # L_cap_x：新字串 (Contraction2欄位)
        cap_x2 = cap_x.replace("'"," ") # L_cap_x2：原始字串 (Final欄位)
        L_cap_x.append(cap_x)
        L_cap_x2.append(cap_x2)
    df_contraction['Contraction2'] = L_cap_x
    df_contraction['Original'] = L_cap_x2
    df_contraction.to_excel('table/df_contraction_edit.xlsx',index=0)
    df_contraction_edit = df_contraction
    return df_contraction_edit,L_cap_x,L_cap_x2

In [158]:
df_contraction_edit,L_cap_x,L_cap_x2 = editWikiTable(df_contraction)
df_contraction_edit

,Contraction,Meaning,Contraction2,Original
0,'aight,alright,'aight,aight
1,ain't,am not / is not / are not / has not / have not...,Ain't,Ain t
2,amn't,am not,Amn't,Amn t
3,aren't,are not,Aren't,Aren t
4,can't,cannot,Can't,Can t
...,...,...,...,...
140,you'll,you shall / you will,You'll,You ll
141,you're,you are/ you are,You're,You re
142,you've,you have,You've,You ve
143,noun's,noun is,Noun's,Noun s


In [159]:
# 自定義建立縮寫替代的字典contractions_dict，L_cap_x2：原始字串，L_cap_x：新字串
# https://github.com/kootenpv/contractions
def create_contractions_dict(L_cap_x2, L_cap_x):
    contractions_dict = dict(zip(L_cap_x2, L_cap_x))
    #print(len(contractions_dict))
    return contractions_dict

In [160]:
contractions_dict = create_contractions_dict(L_cap_x2, L_cap_x)
contractions_dict

{' aight': "'aight",
 'Ain t': "Ain't",
 'Amn t': "Amn't",
 'Aren t': "Aren't",
 'Can t': "Can't",
 ' cause': "'cause",
 'Could ve': "Could've",
 'Couldn t': "Couldn't",
 'Couldn t ve': "Couldn't've",
 'Daren t': "Daren't",
 'Daresn t': "Daresn't",
 'Dasn t': "Dasn't",
 'Didn t': "Didn't",
 'Doesn t': "Doesn't",
 'Don t': "Don't",
 'D ye': "D'ye",
 'E er': "E'er",
 'Everybody s': "Everybody's",
 'Everyone s': "Everyone's",
 'Finna': 'Finna',
 'G day': "G'day",
 'Gimme': 'Gimme',
 'Giv n': "Giv'n",
 'Gonna': 'Gonna',
 'Gon t': "Gon't",
 'Gotta': 'Gotta',
 'Hadn t': "Hadn't",
 'Had ve': "Had've",
 'Hasn t': "Hasn't",
 'Haven t': "Haven't",
 'He d': "He'd",
 'He ll': "He'll",
 'He s': "He's",
 'He ve': "He've",
 'How d': "How'd",
 'Howdy': 'Howdy',
 'How ll': "How'll",
 'How re': "How're",
 'How s': "How's",
 'I d': "I'd",
 'I d ve': "I'd've",
 'I ll': "I'll",
 'I m': "I'm",
 'I m a': "I'm'a",
 'I m o': "I'm'o",
 'Innit': 'Innit',
 'I ve': "I've",
 'Isn t': "Isn't",
 'It d': "It'd",
 'It 

In [161]:
def modify_symbol(path):
    L_allFile = os.listdir(path)
    L_artist = [] # 暫存目前未刪除歌曲的歌手名稱
    L_title = [] # 暫存目前未刪除歌曲的歌曲檔名
    L_original_artist = []
    
    for file in L_allFile:
        if os.path.isdir(os.path.join(path,file)):
            L_artist.append(file)
            
            artistPath = path + '/' + file
            L_artistFile = os.listdir(artistPath)
            
            for file2 in L_artistFile:
                songPath = artistPath + '/' + file2
                if os.path.isfile(songPath):
                    
                    # 若遇到檔案有特殊符號，如「’」、「´」
                    new = file2.replace("’","'").replace("´","'")
                    mp3filename_edit = artistPath + '/' + new
                    os.rename(songPath,mp3filename_edit)
                    
                    L_title.append(new)
                    #print(artistPath + '/' + file2)
                    
                    # 原始歌手資料夾名稱
                    originalname = artistPath.split('/')[-1]
                    L_original_artist.append(originalname)
                    
    df_mp3_1_modify_symbol = pd.DataFrame(list(zip(L_original_artist,L_title)),columns=['ArtistName','SongName'])
    df_mp3_1_modify_symbol.to_excel('table/df_mp3_1_modify_symbol1.xlsx',index=0) # 匯出修改歌曲名稱的符號的xlsx
    return df_mp3_1_modify_symbol

In [162]:
path = r'D:/Anaconda3/Scripts/7 實驗進度/MatchLyrics/songs2' # mp3資料夾位址
df_mp3_1_modify_symbol = modify_symbol(path)
df_mp3_1_modify_symbol

,ArtistName,SongName
0,5 Seconds of Summer,18.mp3
1,5 Seconds of Summer,Airplanes.mp3
2,5 Seconds of Summer,Amnesia.mp3
3,5 Seconds of Summer,Babylon.mp3
4,5 Seconds of Summer,Beside You.mp3
...,...,...
134,Adele,Take It All.mp3
135,Adele,Turning Tables.mp3
136,"Agnetha Fältskog,Gary Barlow",I Should've Followed You Home.mp3
137,"Agnetha Fältskog,Ola Håkansson",The Way You Are - Single Version.mp3


In [163]:
# 判斷字串是否包含子字串：https://blog.csdn.net/yl2isoft/article/details/52079960
# 子字串為自定義，採用建立contractions_dict的L_cap_x2陣列，迴圈一一搜尋
def editSongName(df_mp3_1_modify_symbol,L_cap_x2,df_contraction):
    #artist = 'Adele'
    #title = "Don t Stop"
    new_title_list = []
    
    for j in range(len(df_mp3_1_modify_symbol)):
        title = df_mp3_1_modify_symbol['SongName'][j]
        print(title)
        L_result = []
        
        for i in range(len(L_cap_x2)):
            result = title.find(L_cap_x2[i]) # 回傳為-1代表不存在，回傳不為-1代表存在
            L_result.append(result)
        #print(L_result)

        # 找到list中滿足條件的元素：https://blog.csdn.net/weixin_40995629/article/details/78715517
        # list找出滿足條件的元素位置：https://zhidao.baidu.com/question/488359074223051332.html
        selected = [i for i,v in enumerate(L_result) if v!=-1]
        print(selected) # 印出符合條件的index值

        # 取代原來的歌曲名稱
        if len(selected) != 0:
            num = selected[0] # 取代的新字串在dict中的位置編號
            df_contraction_selected = df_contraction.loc[num] # 透過索引值選擇資料df.loc[[index],[colunm]]
            old_string = df_contraction_selected['Original']
            new_string = df_contraction_selected['Contraction2']
            print(old_string + ' >>> ' + new_string)
            new_title = title.replace(old_string,new_string)
            print(new_title)
            new_title_list.append(new_title)
        else:
            new_title_list.append(title)
            print('None Contraction Find!')
        print('-'*80)
    
    df_mp3_1_modify_symbol['SongNameNew'] = new_title_list
    df_mp3_1_modify_symbol.to_excel('table/df_mp3_1_modify_symbol2.xlsx',index=0) # 修改歌曲名稱後的檔案
    df_mp3_1_modify_symbol_new = df_mp3_1_modify_symbol
    return new_title_list,df_mp3_1_modify_symbol_new

In [164]:
new_title_list,df_mp3_1_modify_symbol_new = editSongName(df_mp3_1_modify_symbol,L_cap_x2,df_contraction)
print(new_title_list)
df_mp3_1_modify_symbol_new

18.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Airplanes.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Amnesia.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Babylon.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Beside You.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Better Man.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Broken Home.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Castaway.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Catch Fire.mp3
[]
None Contraction Find!

[]
None Contraction Find!
--------------------------------------------------------------------------------
I Breathe Again.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
I Can't Live Without Your Love.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
The Best Thing.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Time Is On Our Side.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Touch Me.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
You Make Me Believe In Love.mp3
[]
None Contraction Find!
--------------------------------------------------------------------------------
Take You High.mp3
[]
None Contraction Find!
----------------------------------------------------------------

,ArtistName,SongName,SongNameNew
0,5 Seconds of Summer,18.mp3,18.mp3
1,5 Seconds of Summer,Airplanes.mp3,Airplanes.mp3
2,5 Seconds of Summer,Amnesia.mp3,Amnesia.mp3
3,5 Seconds of Summer,Babylon.mp3,Babylon.mp3
4,5 Seconds of Summer,Beside You.mp3,Beside You.mp3
...,...,...,...
134,Adele,Take It All.mp3,Take It All.mp3
135,Adele,Turning Tables.mp3,Turning Tables.mp3
136,"Agnetha Fältskog,Gary Barlow",I Should've Followed You Home.mp3,I Should've Followed You Home.mp3
137,"Agnetha Fältskog,Ola Håkansson",The Way You Are - Single Version.mp3,The Way You Are - Single Version.mp3


In [165]:
# mp3檔案歌曲名稱重新命名
def rename_mp3file(path,df_mp3_1_modify_symbol_new):
    for i in range(len(df_mp3_1_modify_symbol_new)):
        artistname = df_mp3_1_modify_symbol_new['ArtistName'][i]
        current = df_mp3_1_modify_symbol_new['SongName'][i]
        new = df_mp3_1_modify_symbol_new['SongNameNew'][i]
        currentPath = path + '/' + artistname + '/' + current
        newPath = path + '/' + artistname + '/' + new
        try:
            os.rename(currentPath,newPath)
        except:
            pass
    print('Rename Done!')

In [166]:
rename_mp3file(path,df_mp3_1_modify_symbol_new)

Rename Done!


# ---

In [122]:
# 爬蟲wiki的英文縮寫統整表table
# Wikipedia:List of English contractions (https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions)
# https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722
website_url = requests.get('https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
table_td = table.find_all('td')

# 抓網頁裡的tabe
L_contraction = []
L_meaning = []
for num in range(int(len(table_td)/2)):
    contraction = table_td[2*num].text.replace(' (rarely, cain\'t)','')
    meaning = table_td[2*num+1].text.replace('\n','').replace('[1]','').replace('[2]','').replace('[3]','').replace('[4]','').replace(' (colloquial)','').replace(' (archaic)','').replace(' (colloquial/Southern American English)','').replace(' (possessive forms of many nouns are homographic to this contraction)','')
    L_contraction.append(contraction)
    L_meaning.append(meaning)

df_contraction = pd.DataFrame(list(zip(L_contraction,L_meaning)),columns=['Contraction','Meaning'])
df_contraction.to_excel('table/df_contraction.xlsx',index=0)
df_contraction

,Contraction,Meaning
0,'aight,alright
1,ain't,am not / is not / are not / has not / have not...
2,amn't,am not
3,aren't,are not
4,can't,cannot
...,...,...
140,you'll,you shall / you will
141,you're,you are/ you are
142,you've,you have
143,noun's,noun is


In [124]:
# 添加兩個欄位，Contraction2：首字大寫，Final：「'」取代為空白
L_cap_x = []
L_cap_x2 = []
for i in range(len(df_contraction)):
    x = df_contraction['Contraction'][i]
    cap_x = string.capwords(x) # L_cap_x：新字串 (Contraction2欄位)
    cap_x2 = cap_x.replace("'"," ") # L_cap_x2：原始字串 (Final欄位)
    L_cap_x.append(cap_x)
    L_cap_x2.append(cap_x2)
df_contraction['Contraction2'] = L_cap_x
df_contraction['Original'] = L_cap_x2
df_contraction.to_excel('table/df_contraction2.xlsx',index=0)
df_contraction_edit = df_contraction
df_contraction_edit

,Contraction,Meaning,Contraction2,Original
0,'aight,alright,'aight,aight
1,ain't,am not / is not / are not / has not / have not...,Ain't,Ain t
2,amn't,am not,Amn't,Amn t
3,aren't,are not,Aren't,Aren t
4,can't,cannot,Can't,Can t
...,...,...,...,...
140,you'll,you shall / you will,You'll,You ll
141,you're,you are/ you are,You're,You re
142,you've,you have,You've,You ve
143,noun's,noun is,Noun's,Noun s


In [117]:
# 搜尋特定的值所在列
df_contraction[df_contraction['Original'].isin(['Don t'])]

,Contraction,Meaning,Contraction2,Original
14,don't,do not / does not,Don't,Don t


In [118]:
# 自定義建立縮寫替代的字典contractions_dict，L_cap_x2：原始字串，L_cap_x：新字串
# https://github.com/kootenpv/contractions
contractions_dict = dict(zip(L_cap_x2, L_cap_x))
print(len(contractions_dict))
contractions_dict

145


{' aight': "'aight",
 'Ain t': "Ain't",
 'Amn t': "Amn't",
 'Aren t': "Aren't",
 'Can t': "Can't",
 ' cause': "'cause",
 'Could ve': "Could've",
 'Couldn t': "Couldn't",
 'Couldn t ve': "Couldn't've",
 'Daren t': "Daren't",
 'Daresn t': "Daresn't",
 'Dasn t': "Dasn't",
 'Didn t': "Didn't",
 'Doesn t': "Doesn't",
 'Don t': "Don't",
 'D ye': "D'ye",
 'E er': "E'er",
 'Everybody s': "Everybody's",
 'Everyone s': "Everyone's",
 'Finna': 'Finna',
 'G day': "G'day",
 'Gimme': 'Gimme',
 'Giv n': "Giv'n",
 'Gonna': 'Gonna',
 'Gon t': "Gon't",
 'Gotta': 'Gotta',
 'Hadn t': "Hadn't",
 'Had ve': "Had've",
 'Hasn t': "Hasn't",
 'Haven t': "Haven't",
 'He d': "He'd",
 'He ll': "He'll",
 'He s': "He's",
 'He ve': "He've",
 'How d': "How'd",
 'Howdy': 'Howdy',
 'How ll': "How'll",
 'How re': "How're",
 'How s': "How's",
 'I d': "I'd",
 'I d ve': "I'd've",
 'I ll': "I'll",
 'I m': "I'm",
 'I m a': "I'm'a",
 'I m o': "I'm'o",
 'Innit': 'Innit',
 'I ve': "I've",
 'Isn t': "Isn't",
 'It d': "It'd",
 'It 

In [139]:
# 判斷字串是否包含子字串：https://blog.csdn.net/yl2isoft/article/details/52079960
# 子字串為自定義，採用建立contractions_dict的L_cap_x2陣列，迴圈一一搜尋
artist = 'Adele'
title = "Don t Stop"
L_result = []
for i in range(len(L_cap_x2)):
    result = title.find(L_cap_x2[i]) # 回傳為-1代表不存在，回傳不為-1代表存在
    L_result.append(result)
print(L_result)

# 找到list中滿足條件的元素：https://blog.csdn.net/weixin_40995629/article/details/78715517
# list找出滿足條件的元素位置：https://zhidao.baidu.com/question/488359074223051332.html
selected = [i for i,v in enumerate(L_result) if v!=-1]
print(selected) # 印出符合條件的index值

# 取代原來的歌曲名稱
if len(selected) != 0:
    num = selected[0] # 取代的新字串在dict中的位置編號
    df_contraction_selected = df_contraction.loc[num] # 透過索引值選擇資料df.loc[[index],[colunm]]
    old_string = df_contraction_selected['Original']
    new_string = df_contraction_selected['Contraction2']
    print(old_string + ' >>> ' + new_string)
    new_title = title.replace(old_string,new_string)
    print(new_title)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[14]
Don t >>> Don't
Don't Stop


In [1]:
# https://stackoverflow.com/questions/43018030/replace-apostrophe-short-words-in-python
# https://github.com/kootenpv/contractions
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [2]:
text="What's the best way to ensure this?"
for word in text.split():
    if word.lower() in contractions:
        text = text.replace(word, contractions[word.lower()])
print(text)

what has / what is the best way to ensure this?


In [3]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
test = "Hey I'm Yann, how're you and how's it going ? That's interesting: I'd love to hear more about it."
print(decontracted(test))

Hey I am Yann, how are you and how is it going ? That is interesting: I would love to hear more about it.
